In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import sys
print('Python: {}'.format(sys.version))

import scipy
print('scipy: {}'.format(scipy.__version__))

import csv
import numpy as np
import sklearn
from sklearn.metrics import mean_squared_error
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import (PCA, LatentDirichletAllocation)
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.feature_selection import f_regression
from sklearn.svm import SVC
from sklearn.ensemble import (RandomForestClassifier, ExtraTreesClassifier, RandomForestRegressor, GradientBoostingClassifier )
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import make_scorer
import xgboost as xgb
from sklearn.impute import SimpleImputer


import random as rn
from biosppy.signals import (ecg, tools)
import pywt
from tqdm import tqdm_notebook as tqdm

import math
from itertools import product
# ============= CONSTS =============
TRAIN_FILE_PATH = "X_train.csv"
TARGET_FILE_PATH =  "y_train.csv"
TEST_FILE_PATH = "X_test.csv"
SAMPLE_FILE_PATH = "sample.csv"

seed = 42
NUM_MAX_COLS = 18154
SAMPLING_RATE=300
USE_WAVE_LETS = False
#my_cols = ["id"] + ["x" + str(i) for i in range(NUM_MAX_POINTS)]
# ============= CONSTS =============

np.random.seed(seed)
rn.seed(seed)

Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
scipy: 1.3.0


# Loading Data

In [2]:
xtrain = pd.read_csv(TRAIN_FILE_PATH)
xtrain.drop("id", axis=1, inplace=True)

ytrain = pd.read_csv(TARGET_FILE_PATH)
ytrain.drop("id", axis=1, inplace = True)

xtest =  pd.read_csv(TEST_FILE_PATH)
id_test = xtest.columns[0]
xtest.drop("id", axis=1, inplace=True)


In [3]:
print(xtrain.shape)
print(ytrain.shape)
print(xtest.shape)

(5117, 17813)
(5117, 1)
(3411, 17813)


# Define functions -- Submission and Feature Extraction

In [4]:
def mean_sqrd_diff(rpeaks):
    diff = np.diff(rpeaks)
    mean_sqrd = np.mean(diff*diff)
    
    return mean_sqrd 

def make_submission(filename, predictions):
    sample =  pd.read_csv(SAMPLE_FILE_PATH)
    sample["y"] = predictions
    sample.to_csv(filename, index= False)

def get_features_from_raw_qrs(signal, sampling_rate):
    X = list()
    ts, filtered, rpeaks, templates_ts, templates, heart_rate_ts, heart_rate = ecg.ecg(signal, sampling_rate, show = False)
    
    '''
    Correct R-peak locations to the maximum within a tolerance
    '''
    rpeaks = ecg.correct_rpeaks(signal = signal, rpeaks=rpeaks, sampling_rate = sampling_rate, tol=0.01)  
    
    '''
    extracting values of R-peaks -- Note: rpeaks gives only indices for R-peaks location
    '''
    peaks = signal[rpeaks]
    
    if len(heart_rate) < 2:
        heart_rate = [0, 1]
    if len(heart_rate_ts) < 2:
        heart_rate_ts = [0, 1]
           
    X.append(np.mean(peaks))
    X.append(np.median(peaks))
    X.append(np.min(peaks))
    X.append(np.max(peaks))
    X.append(np.std(peaks))
    X.append(np.mean(rpeaks))
    X.append(np.median(rpeaks))
    X.append(np.min(rpeaks))
    X.append(np.max(rpeaks))
    X.append(np.std(rpeaks))
    X.append(np.sqrt(mean_sqrd_diff(rpeaks)))        ## remove if results worsen
    X.append(np.mean(np.diff(rpeaks)))
    X.append(np.median(np.diff(rpeaks)))
    X.append(np.min(np.diff(rpeaks)))
    X.append(np.max(np.diff(rpeaks)))
    X.append(np.std(np.diff(rpeaks)))           
    X.append(np.mean(heart_rate))
    X.append(np.median(heart_rate))
    X.append(np.min(heart_rate))
    X.append(np.max(heart_rate))
    X.append(np.std(heart_rate))
    X.append(np.mean(np.diff(heart_rate)))
    X.append(np.median(np.diff(heart_rate)))
    X.append(np.min(np.diff(heart_rate)))
    X.append(np.max(np.diff(heart_rate)))
    X.append(np.std(np.diff(heart_rate)))
    X.append(np.mean(heart_rate_ts))
    X.append(np.median(heart_rate_ts))
    X.append(np.min(heart_rate_ts))
    X.append(np.max(heart_rate_ts))
    X.append(np.std(heart_rate_ts))
    X.append(np.mean(np.diff(heart_rate_ts)))
    X.append(np.median(np.diff(heart_rate_ts)))
    X.append(np.min(np.diff(heart_rate_ts)))
    X.append(np.min(np.diff(heart_rate_ts)))
    X.append(np.max(np.diff(heart_rate_ts)))
    X.append(np.std(np.diff(heart_rate_ts)))
    X.append(np.sum(filtered-signal))
    
    X += list(np.mean(templates, axis=0))
    X += list(np.median(templates, axis=0))
    X += list(np.std(templates, axis=0))
    #X += list(np.abs(np.fft.rfft(np.mean(templates, axis=0), axis=0))[0:45])   ### adding FFT (choose only half of entries)
    X += list(np.min(templates, axis=0))
    X += list(np.max(templates, axis=0))
    X = np.array(X)
    
    #X[np.isnan(X)] = 0
    return X

# Obtain features from raw signal

In [5]:
features = list()
sampling_rate = float(SAMPLING_RATE)
for id in tqdm(range(xtrain.shape[0])):
    signal = np.array(pd.to_numeric(xtrain.iloc[id].dropna()))
    features.append(get_features_from_raw_qrs(signal, sampling_rate))
    
    
X = np.array(features)
y = np.ravel(np.array(ytrain.values))

features_test = list()
for id in tqdm(range(xtest.shape[0])):
    signal = np.array(pd.to_numeric(xtest.iloc[id].dropna()))
    features_test.append(get_features_from_raw_qrs(signal, sampling_rate))
    
X_test = np.array(features_test)

# Model Selection 

In [6]:
print(X.shape)
print(y.shape)
print(X_test.shape)

(5117, 938)
(5117,)
(3411, 938)


In [253]:
'''
create subset of initial dataframe X for model selection 
'''
X = pd.DataFrame(X) 
X['y'] = y
X_sub = pd.DataFrame(X).sample(frac = 0.40, replace = False, axis=0)
y_sub = X_sub['y']
X_sub = X_sub.drop('y', axis = 1).values
X = X.drop('y', axis = 1)
print(y_sub.shape, X_sub.shape)

(2047,) (2047, 654)


## Notes on models:

Best results so far: 

0.7787005373717636 (2 fold GS CV) -- Model: Gradient_boosting_classifier
0.825726141079	public leaderboard
{'classifier__learning_rate': 0.05, 'classifier__max_depth': 5, 'classifier__max_features': 40, 'classifier__n_estimators': 200}
No fft included in features 

Next to run: 
same as above with fft included in features

In [7]:
scorer_f1 = make_scorer(f1_score, greater_is_better=True, average='micro')

'''
Gradient Boosting APPROACH -- GRID-SEARCH CV
Run with added FFT
''' 

steps = [("impute", SimpleImputer()),
         ("scaler", preprocessing.StandardScaler()), 
         ("classifier", GradientBoostingClassifier())]
pipeline = Pipeline(steps = steps)

parameters = {"impute__strategy": ["mean", "median", "constant"],
              "impute__fill_value": [0],
              "classifier__max_depth": [3,4,5,6,7,8],
              "classifier__n_estimators": [200,250,300],
              "classifier__learning_rate": [0.1,0.08,0.05,0.03],
              "classifier__max_features": [40,50,60]
             }

grid = GridSearchCV(pipeline, parameters, cv = 2, scoring = scorer_f1, verbose = 1)

grid.fit(X, y)
print(grid.best_score_)
print(grid.best_params_)


'''
## SVC APPROACH -- GRID-SEARCH CV

steps = [("scaler", preprocessing.StandardScaler()), ("classifier", SVC())]
pipeline = Pipeline(steps = steps)

parameters = {"classifier__kernel": ["rbf", "poly"],
              "classifier__gamma": ["auto"],
              "classifier__C": [15,30,45,60,75],  
              "classifier__class_weight": ["balanced"],
              "classifier__degree": [2,4,6,8]
             }

grid = GridSearchCV(pipeline, parameters, cv = 5, scoring = scorer_f1, verbose = 2)

grid.fit(X, y)
print(grid.best_score_)
print(grid.best_params_)

estimator = SVC(C = grid.best_params_['classifier__C'], gamma = 'auto', 
                class_weight = 'balanced', 
                kernel = grid.best_params_['classifier__kernel'], 
                degree = grid.best_params_['classifier__degree'])

estimator.fit(xtrain_scaled, y)
pred = estimator.predict(xtest_scaled)
make_submission("prediction_trial.csv", pred)
'''

'''
## XGB APPROACH -- GRID-SEARCH CV


steps = [("scaler", preprocessing.StandardScaler()), ("classifier", xgb.XGBClassifier())]
pipeline = Pipeline(steps = steps)

parameters = {"classifier__max_depth": [5,10,15],
              "classifier__n_estimators": [200],
              "classifier__learning_rate": [0.05,0.1],
              "classifier__max_features": [20,40]
             }

grid = GridSearchCV(pipeline, parameters, cv = 2, scoring = scorer_f1, verbose = 1)

grid.fit(X_sub, y_sub)
print(grid.best_score_)
print(grid.best_params_)

'''

Fitting 2 folds for each of 648 candidates, totalling 1296 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1296 out of 1296 | elapsed: 577.2min finished


0.8151260504201681
{'classifier__learning_rate': 0.1, 'classifier__max_depth': 5, 'classifier__max_features': 60, 'classifier__n_estimators': 250, 'impute__fill_value': 0, 'impute__strategy': 'median'}


'\n## XGB APPROACH -- GRID-SEARCH CV\n\n\nsteps = [("scaler", preprocessing.StandardScaler()), ("classifier", xgb.XGBClassifier())]\npipeline = Pipeline(steps = steps)\n\nparameters = {"classifier__max_depth": [5,10,15],\n              "classifier__n_estimators": [200],\n              "classifier__learning_rate": [0.05,0.1],\n              "classifier__max_features": [20,40]\n             }\n\ngrid = GridSearchCV(pipeline, parameters, cv = 2, scoring = scorer_f1, verbose = 1)\n\ngrid.fit(X_sub, y_sub)\nprint(grid.best_score_)\nprint(grid.best_params_)\n\n'

# Make Submission

In [8]:
impute = SimpleImputer(strategy = 'median', fill_value = 0)
X = impute.fit_transform(X)
X_test = impute.fit_transform(X_test)

scaler = StandardScaler() 
scaler.fit(X)
x_train_scaled = scaler.transform(X)
x_test_scaled = scaler.transform(X_test)


'''
change based on the output of the above
'''
estimator = GradientBoostingClassifier(n_estimators = 250, 
                                       max_depth = 5,
                                       learning_rate = 0.1, 
                                       max_features= 60)


'''
estimator = GradientBoostingClassifier(n_estimators = grid.best_params_['classifier__n_estimators'], 
                                       max_depth = grid.best_params_['classifier__max_depth'],
                                       learning_rate = grid.best_params_['classifier__learning_rate'], 
                                       max_features= grid.best_params_['classifier__max_features'])
'''

estimator.fit(x_train_scaled, y)
pred = estimator.predict(x_test_scaled)
make_submission("prediction_gradient_booster_even_more_feats.csv", pred)